In [1]:
import json
import numpy as np
import sys
import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [2]:
dev_rs_logits = json.load(open('/mnt/infonas/data/yashgupta/data/row_selector_output/qid_logits_bert_large_dev.json'))

In [3]:
dev_rc_logits = json.load(open('/tmp/nbest_predictions_dev_top5.json'))

In [4]:
eval_scores = json.load(open('/tmp/dev_top5_scores.json'))

In [5]:
def get_rs_score(q_id, rank):
    return np.sort(np.array(dev_rs_logits[q_id]))[-5:][rank]

X, y = [], []
for k, v in dev_rc_logits.items():
    feats = []
    qid, rank = k.split('_')
    feats.append(get_rs_score(qid, int(rank)))
    # print(v)
    feats.append(v[0]['start_logit'])
    feats.append(v[0]['end_logit'])
    X.append(feats)
    y.append(eval_scores[k]['em-score'])

In [6]:
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(17330, 3) (17330,)


In [7]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

In [8]:
clf.fit(X, y)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [9]:
clf.predict_proba([[0, 0, 0]])

array([[0.5105333, 0.4894667]])

In [10]:
clf.score(X, y)

0.8691863819965377

In [11]:
from tqdm.notebook import tqdm
fnd = {}
for k, v in tqdm(dev_rc_logits.items()):
    feats = []
    qid, rank = k.split('_')
    feats.append(get_rs_score(qid, int(rank)))
    # print(v)
    feats.append(v[0]['start_logit'])
    feats.append(v[0]['end_logit'])
    prob = clf.predict_proba([feats])[0][1]
    if qid not in fnd.keys():
        fnd[qid] = [prob, rank, eval_scores[k]['em-score'], eval_scores[k]['f1-score']]
    elif fnd[qid][0] < prob:
         fnd[qid] = [prob, rank, eval_scores[k]['em-score'], eval_scores[k]['f1-score']]

  0%|          | 0/17330 [00:00<?, ?it/s]

In [12]:
len(fnd)

3466

In [14]:
ems = np.array([x[2] for x in fnd.values()])
f1s = np.array([x[3] for x in fnd.values()])

In [15]:
ems.mean()

0.5874206578188114

In [17]:
f1s.mean()

0.6561356874701547

In [18]:
clf.feature_importances_

array([0.42, 0.28, 0.3 ])

In [19]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
regr = AdaBoostRegressor(random_state=0, n_estimators=100)

In [20]:
X, y = [], []
for k, v in dev_rc_logits.items():
    feats = []
    qid, rank = k.split('_')
    feats.append(get_rs_score(qid, int(rank)))
    # print(v)
    feats.append(v[0]['start_logit'])
    feats.append(v[0]['end_logit'])
    X.append(feats)
    y.append(eval_scores[k]['f1-score'])

In [21]:
regr.fit(X, y)

AdaBoostRegressor(n_estimators=100, random_state=0)

In [23]:
regr.score(X, y)

0.40254726245483985

In [25]:
regr.predict([[0, 0, 0]])

array([0.13732496])

In [26]:
from tqdm.notebook import tqdm
fnd = {}
for k, v in tqdm(dev_rc_logits.items()):
    feats = []
    qid, rank = k.split('_')
    feats.append(get_rs_score(qid, int(rank)))
    # print(v)
    feats.append(v[0]['start_logit'])
    feats.append(v[0]['end_logit'])
    prob = regr.predict([feats])[0]
    if qid not in fnd.keys():
        fnd[qid] = [prob, rank, eval_scores[k]['em-score'], eval_scores[k]['f1-score']]
    elif fnd[qid][0] < prob:
         fnd[qid] = [prob, rank, eval_scores[k]['em-score'], eval_scores[k]['f1-score']]

  0%|          | 0/17330 [00:00<?, ?it/s]

In [27]:
ems = np.array([x[2] for x in fnd.values()])
f1s = np.array([x[3] for x in fnd.values()])

In [28]:
ems.mean()

0.5614541257934218

In [29]:
f1s.mean()

0.6327459560885017